## Initialization  

In [1]:
%load_ext autoreload
%autoreload 2
import os 
import sys
sys.path.insert(0, '../src')
import time
import argparse
import yaml
import types
import copy, pprint
from time import sleep
from datetime import datetime
import numpy  as np
import torch  
import wandb
from pynvml import *
import pandas as pd
from utils.notebook_modules import (initialize, init_dataloaders, init_environment, init_wandb, 
                                   training_prep, disp_dataloader_info,disp_info_1, 
                                   warmup_phase, weight_policy_training, display_gpu_info,
                                   init_dataloaders_by_fold_id)
                                    
from utils import (print_separator, print_heading, timestring, print_loss, load_from_pickle) #, print_underline, 
#                       print_dbg, get_command_line_args ) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
# print(sys.path)
# disp_gpu_info() 
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train_mini.ipynb"

display_gpu_info()



cuda is available
 CUDA device count  :  1
 CUDA current device:  0
 GPU Processes      : 
 GPU:0
no processes are running
 Device : cuda:0
   name:        NVIDIA GeForce GTX 970M
   capability:  (5, 2)
   properties:  _CudaDeviceProperties(name='NVIDIA GeForce GTX 970M', major=5, minor=2, total_memory=3071MB, multi_processor_count=10)
   Allocated :  0
   Reserved  :  0

| ID | GPU  | MEM |
-------------------
|  0 | nan% |  1% |
cuda:0
0
nvml_gpu_id:  0
c_nvmlMemory_t(total: 3221225472 B, free: 3179544576 B, used: 41680896 B)


## Create Environment

### Parse Input Args  - Read YAML config file - wandb initialization

In [2]:
# RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

## For RESTARTING
##
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --exp_desc    Train with dropout 0.5" \
#              " --seed_idx    0 "\
#              " --batch_size  128" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
## get command line arguments

In [2]:
synthetic_1task_config = "../yamls/chembl_synt_train_1task.yaml"
synthetic_3task_config = "../yamls/chembl_synt_train_3task.yaml"
synthetic_5task_config = "../yamls/chembl_synt_train_5task.yaml"
synthetic_config = "../yamls/chembl_synt_train.yaml"
mini_config      = "../yamls/chembl_mini_train.yaml"
##  For Initiating 
##
input_args = f" --config  {mini_config} " \
             " --exp_desc     weight 105 bch/ep policy 105 bch/ep " \
             " --warmup_epochs        10 "  \
             " --hidden_size          1000 "  \
             " --tail_hidden_size     1000 "  \
             " --first_dropout       0.45 "  \
             " --middle_dropout      0.45"  \
             " --last_dropout        0.45 "  \
             " --seed_idx              0 "  \
             " --batch_size          128 "  \
             " --task_lr           0.001 "  \
             " --backbone_lr       0.001 "  \
             " --decay_lr_rate       0.3 "  \
             " --decay_lr_freq        10 "  \
             " --policy_lr         0.001 "  \
             " --lambda_sparsity    0.02 "  \
             " --lambda_sharing     0.01 "  \
             " --skip_hidden " \
             " --cpu " 

#              " --no_residual "
#              " --exp_name       0410_1934 " \
#              " --hidden_size   100 100 100 100 100 100" \
#              " --tail_hidden_size  100 " \
#              " --decay_lr_rate      0.75"  \
#              " --decay_lr_freq       20"  \


In [3]:
opt, ns = initialize(input_args, build_folders = True)

wandb: Currently logged in as: kbardool (use `wandb login --relogin` to force relogin)



  command line parms : 
------------------------
 config...................  ../yamls/chembl_mini_train.yaml
 exp_id...................  1z349wb6
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  weight 105 bch/ep policy 105 bch/ep
 hidden_sizes.............  [1000]
 tail_hidden_size.........  [1000]
 warmup_epochs............  10
 training_epochs..........  None
 seed_idx.................  0
 batch_size...............  128
 first_dropout............  0.45
 middle_dropout...........  0.45
 last_dropout.............  0.45
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.001
 decay_lr_rate............  0.3
 decay_lr_freq............  10.0
 lambda_sparsity..........  0.02
 lambda_sharing...........  0.01
 gpu_ids..................  [0]
 skip_residual............  False
 skip_hidden..............  True
 resume...................  False
 cpu......................  True
 min_samples_class........  N

error: cannot run ipynb_drop_output: No such file or directory
error: cannot fork to run external filter 'ipynb_drop_output'
error: external filter 'ipynb_drop_output' failed
error: cannot run ipynb_drop_output: No such file or directory
error: cannot fork to run external filter 'ipynb_drop_output'
error: external filter 'ipynb_drop_output' failed


 PROJECT NAME: AdaSparseChem-Mini
 RUN ID      : 1z349wb6 
 RUN NAME    : 0430_1951_skip_hdn_skip_hdn
 PROJECT NAME: AdaSparseChem-Mini
 RUN ID      : 1z349wb6 
 RUN NAME    : 0430_1951_skip_hdn_skip_hdn


 log_dir              create folder:  ../../experiments/mini-AdaSparseChem/1000x1_0430_1951_lr0.001_do0.45_skip_hdn
 result_dir           folder exists:  ../../experiments/mini-AdaSparseChem/1000x1_0430_1951_lr0.001_do0.45_skip_hdn
 checkpoint_dir       folder exists:  ../../experiments/mini-AdaSparseChem/1000x1_0430_1951_lr0.001_do0.45_skip_hdn

------------------------------------------------------------------------------------------------------------------------
 experiment name       : 0430_1951_skip_hdn_skip_hdn 
 experiment id         : 1z349wb6 
 folder_name           : 1000x1_0430_1951_lr0.001_do0.45_skip_hdn 
 experiment description: weight 105 bch/ep policy 105 bch/ep
 Random seeds          : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
 Random  seed used     : 88 
 log fold

In [5]:
# ns.wandb_run.finish()

In [34]:
# dldrs = init_dataloaders(opt, verbose = False)
dldrs = init_dataloaders_by_fold_id(opt, verbose = True)

----------------------------------------------------------------
 2022-04-30 21:20:48:890014  Create new  Chembl_23 instance 
---------------------------------------------------------------- 

 verbose        : True

 FOLDS param provided - folds: [1, 2, 3, 4] 

 X (ecfp[0]) file count non zero (post fold & transform) :80 


 Load label/Y file for task 1 filename : chembl_23mini_y.npy . . .
-------------------------------------------------------------------
 load_task_weights() - filename: None label: y_task1
 load_task_weights() - no weights file provided for y_task1, training_weights for all  100 classes set to 1 
 Number of non-zero features in ecfp[0]:80

 Task 1 label file: 
    Total Positive Labels :     18704 
    Total Negative Labels :      2445 
    Total Non-Zero Labels :     21149
 All y values are 0, 1, or -1.

 tasks_class.aggregation_weight WAS NOT passed 
 min_samples_class: 1

 Class fold counts: 
---------------------
 fold_pos - total: 18704 - .sum(axis=0):
[ 884   

In [35]:
disp_dataloader_info(dldrs)


 trainset.y_class                                   :  [(14633, 100)] 
 trainset1.y_class                                  :  [(14633, 100)] 
 trainset2.y_class                                  :  [(14633, 100)] 
 valset.y_class                                     :  [(3755, 100)]  
                                 
 size of training set 0 (warm up)                   :  14633 
 size of training set 1 (network parms)             :  14633 
 size of training set 2 (policy weights)            :  14633 
 size of validation set                             :  3755 
                               Total                :  47654 
                                 
 Number of batches in training 0 (warm up)          :  115 
 Number of batches in training 1 (network parms)    :  115 
 Number of batches in training 2 (policy weights)   :  115 
 Number of batches in validation dataset            :  30 
                                


In [36]:
print("validation set ", dldrs.valset.num_neg.sum())
print("validation set ", dldrs.valset.num_pos.sum())
print("training set ", dldrs.trainset0.num_neg.sum())
print("training set ", dldrs.trainset0.num_pos.sum())

validation set  2445
validation set  18704
training set  2445
training set  18704


In [ ]:
for i in zip(dldrs.valset.num_pos, dldrs.valset.num_neg, dldrs.trainset0.num_pos, dldrs.trainset0.num_neg):
    print(f" {i[0]:4d}  {i[1]:4d}    trianing: {i[2]:4d}   {i[3]:4d}")

In [37]:
print(dldrs.trainset0.tasks_weights_list[0].aggregation_weight.sum())
print(dldrs.valset.tasks_weights_list[0].aggregation_weight.sum())

20.0
20.0


### Setup Dataloader and Model  

In [38]:
environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = True)
# environ.define_optimizer(policy_learning=False)
# environ.define_scheduler(policy_learning=False)

# environ.optimizers['weights'].param_groups[0]
# print(environ.print_configuration())

##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------
 SparseChem_Backbone  Ver: 1.0 Init() Start 
-------------------------------------------------- 

 layer config        : [1] 
 skip residual layers: False   skip hidden layers  : True
 SparseChem_BackBone() Input Layer  - Input: 32000  output: 1000  non-linearity:<class 'torch.nn.modules.activation.ReLU'>
 Module List 
--------------------------------------------------
 Initialize weights 
-------------------------------------------------- 

------------------------------------------------------------------------------------------------------------------------
 SparseChem Backbone -- Final configuration(2) : 
                     self.blocks: <class 'torch.nn.modules.container.ModuleList'>  len:0
-----------------------------------------------------------------------

### Initiate / Resume Training Prep

In [39]:
# ns.wandb_run.finish()
# check_for_resume_training(ns, opt, environ)

### Training Preparation

In [40]:
training_prep(ns, opt, environ, dldrs)

# print('-'*80)
# disp_info_1(ns, opt, environ)

print('-'*80)
print(environ.disp_for_excel())

 training preparation: - check for CUDA - cuda available as device id: [0]
 training preparation: - set print_freq to length of train loader: 115
 training preparation: - set eval_iters to length of val loader : 30
 training preparation: - set number of batches per weight training epoch to: 115
 training preparation: - set number of batches per policy training epoch to: 115
--------------------------------------------------------------------------------

    folder: 1000x1_0430_1951_lr0.001_do0.45_skip_hdn
    layers: 1 [1000] 
    
    first dropout          : 0.45
    middle dropout         : 0.45
    last dropout           : 0.45
    diff_sparsity_weights  : False
    skip_layer             : 0
    is_curriculum          : False
    curriculum_speed       : 3
    
    task_lr                : 0.001
    backbone_lr            : 0.001
    decay_lr_rate          : 0.3
    decay_lr_freq          : 10.0
    
    policy_lr              : 0.001
    policy_decay_lr_rate   : 0.75
    policy_

## Warmup Training

In [41]:
# environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
# ns.check_for_improvment_wait = 0
ns.warmup_epochs = 50
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warmup_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.warmup_epochs}", verbose = True)

------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  50 - Run epochs 1 to 50
------------------------------------------------------------------------ 



In [ ]:

warmup_phase(ns,opt, environ, dldrs)

print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best Precision :   {ns.best_value:.5f}\n")
print()

------------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  50 - Run epochs 1 to 50
------------------------------------------------------------------------ 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |   logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total | time |
   1 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   0.1025   3.380e-04   0.000e+00    0.1028 |   0.00002   0.25305   0.94047   0.88928   0.93708   0.94552 |   0.0989   1.154e-04   0.000e+00    0.0990 | 30.9 |
Previous best_epoch:     0   best iter:     0,   best_value: 0.00000
New      best_epoch:     1   best iter:   115,   best_value: 0.94047
   2 | 1.00e-03  1.00e-03  1.00e-03  4.00e+00 |   0.0230   3.380e-04   0.000e+00    0.0233 |   0.00002   0.22762   0.91136   0.90196   0.90011   0.91448 |   0.0842   1.154e-04   0.000e+00    0.0843 | 31.2 |
   3 | 1.00e-03  1.00e-0

  35 | 3.00e-04  3.00e-04  1.00e-03  4.00e+00 |   0.0025   3.380e-04   0.000e+00    0.0029 |   0.00002   0.26382   0.92591   0.91248   0.91138   0.92402 |   0.0971   1.154e-04   0.000e+00    0.0972 | 29.2 |
  36 | 3.00e-04  3.00e-04  1.00e-03  4.00e+00 |   0.0119   3.380e-04   0.000e+00    0.0122 |   0.00002   0.30106   0.94998   0.91981   0.94832   0.93951 |   0.0951   1.154e-04   0.000e+00    0.0952 | 31.4 |
  37 | 3.00e-04  3.00e-04  1.00e-03  4.00e+00 |   0.0012   3.380e-04   0.000e+00    0.0015 |   0.00002   0.28423   0.92723   0.91053   0.91304   0.92660 |   0.0967   1.154e-04   0.000e+00    0.0968 | 30.0 |
Epoch    37: reducing learning rate of group 0 to 9.0000e-05.
Epoch    37: reducing learning rate of group 1 to 9.0000e-05.
  38 | 9.00e-05  9.00e-05  1.00e-03  4.00e+00 |   0.0012   3.380e-04   0.000e+00    0.0015 |   0.00002   0.29054   0.92659   0.91225   0.91272   0.92422 |   0.0957   1.154e-04   0.000e+00    0.0959 | 31.7 |
  39 | 9.00e-05  9.00e-05  1.00e-03  4.00e+00 | 

In [12]:
print(f"Best Epoch :       {ns.best_epoch}\n"
      f"Best Iteration :   {ns.best_iter} \n"
      f"Best Precision :   {ns.best_value:.5f}\n")
print()
pp.pprint(environ.val_metrics['aggregated'])

Best Epoch :       23
Best Iteration :   2645 
Best Precision :   0.94657


{   'auc_pr': 0.9357182859417005,
    'avg_prec_score': 0.9429960304326884,
    'bceloss': 0.38601235412538815,
    'f1_max': 0.9398995764862556,
    'kappa': 0.4139156231753311,
    'kappa_max': 0.7667071541394691,
    'logloss': 2.1372066841553236e-05,
    'p_f1_max': 0.73216563440525,
    'p_kappa_max': 0.8152552059827707,
    'roc_auc_score': 0.8845997060665461,
    'sc_loss': 0.0030697412006750963}


In [14]:
# pp.pprint(environ.val_metrics)
df = environ.val_metrics['task1']['classification']

In [15]:
print(df[pd.notna(df.roc_auc_score)])

      roc_auc_score    auc_pr  avg_prec_score    f1_max  p_f1_max     kappa  kappa_max  p_kappa_max   bceloss
task                                                                                                         
0          0.989003  0.998409        0.998381  0.977974  0.967152  0.729923   0.843007     0.967152  0.105100
5          0.944444  0.996995        0.997076  0.972973  0.479137 -0.055556   0.641509     0.990281  0.124514
6          0.998358  0.999976        0.999976  0.997543  0.977779  0.797642   0.854725     0.988166  0.020145
7          0.833333  0.850000        0.866667  0.800000  0.982072  0.000000   0.695652     0.982072  1.264657
8          1.000000  1.000000        1.000000  1.000000  0.981241  0.000000   1.000000     0.981241  0.097502
9          1.000000  1.000000        1.000000  1.000000  0.998768  0.660870   1.000000     0.998768  0.016954
14         0.993333  0.999823        0.999825  0.993377  0.966154  0.000000   0.793566     0.994513  0.067134
19        

In [16]:
df[pd.notna(df.roc_auc_score)].mean()

roc_auc_score     0.884600
auc_pr            0.935718
avg_prec_score    0.942996
f1_max            0.939900
p_f1_max          0.732166
kappa             0.413916
kappa_max         0.766707
p_kappa_max       0.815255
bceloss           0.386012
dtype: float64

In [15]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)
ns.wandb_run.finish()

# ns.wandb_run.finish()
# environ.losses
# environ.val_metrics

auc_pr,▁
avg_prec_score,▁
bceloss,▁
best_accuracy,▁
best_epoch,▁
best_iter,▁
epoch,▁
f1_max,▁
gumbel_temp,▁
kappa,▁
kappa_max,▁


In [16]:
ns.check_for_improvment_wait
ns.curriculum_epochs

0

## Check values

In [17]:
pp.pprint(environ.val_metrics)

{   'aggregated': {   'auc_pr': 0.9357182859417005,
                      'avg_prec_score': 0.9429960304326884,
                      'bceloss': 0.38601235412538815,
                      'f1_max': 0.9398995764862556,
                      'kappa': 0.4139156231753311,
                      'kappa_max': 0.7667071541394691,
                      'logloss': 2.1372066841553236e-05,
                      'p_f1_max': 0.73216563440525,
                      'p_kappa_max': 0.8152552059827707,
                      'roc_auc_score': 0.8845997060665461,
                      'sc_loss': 0.0030697412006750963},
    'epoch': 50,
    'parms': {   'gumbel_temp': 4,
                 'lambda_sharing': 0.01,
                 'lambda_sparsity': 0.02,
                 'lambda_tasks': 1,
                 'lr_0': 8.999999999999999e-05,
                 'lr_1': 8.999999999999999e-05,
                 'policy_lr': 0.001,
                 'train_layers': 0},
    'sharing': {'total': 0.0},
    'sparsity': {   't

In [16]:
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.num_tasks
# print(environ.get_policy_prob().shape)
# print(environ.val_data['task1'].keys())
# print(environ.val_data['task1']['yc_ind'][0][:40])
# print(environ.val_data['task1']['yc_ind'][1][:40])
# print(environ.val_data['task1']['yc_data'][:40])
# print(environ.val_data['task1']['yc_hat'][:40])
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.display_trained_logits(ns.current_epoch,out=[sys.stdout])

In [17]:
# pp.pprint(environ.val_metrics['task1'])
pp.pprint(environ.val_metrics)
# pp.pprint(ns.val_metrics)
# print((environ.val_data['task1']['yc_data']).sum())
# print(len(environ.val_data['task1']['yc_ind'][1]))
# print(len(environ.val_data['task1']['yc_data']))
# print(len(environ.val_data['task1']['yc_hat']))

{   '_runtime': 58,
    '_timestamp': 1649841795,
    'aggregated': {   'auc_pr': 0.8863761813736045,
                      'avg_prec_score': 0.9038057874228314,
                      'bceloss': 0.5405514872601578,
                      'f1_max': 0.9285540758241476,
                      'kappa': 0.4061207927257367,
                      'kappa_max': 0.7299278430897939,
                      'logloss': 3.2356826155563767e-05,
                      'p_f1_max': 0.6895967995007641,
                      'p_kappa_max': 0.7860043613383403,
                      'roc_auc_score': 0.8293258801651597,
                      'sc_loss': 0.004647518796810809},
    'epoch': 25,
    'parms': {   'gumbel_temp': 4,
                 'lambda_sharing': 0.01,
                 'lambda_sparsity': 0.02,
                 'lambda_tasks': 1,
                 'lr_0': 0.001,
                 'lr_1': 0.001,
                 'policy_lr': 0.001,
                 'train_layers': 0},
    'sharing': {'total': 0.0},
    

In [16]:
type(ns.val_metrics['aggregated'])

dict

In [23]:
pp.pprint(ns.trn_losses)

{   '_runtime': 344,
    '_timestamp': 1649839198,
    'parms': {   'gumbel_temp': 4,
                 'lambda_sharing': 0.01,
                 'lambda_sparsity': 0.02,
                 'lambda_tasks': 1,
                 'lr_0': 7.289999999999998e-07,
                 'lr_1': 7.289999999999998e-07,
                 'policy_lr': 0.001,
                 'train_layers': 0},
    'sharing': {'total': tensor(0.)},
    'sparsity': {'task1': tensor(0.000338), 'total': tensor(0.000338)},
    'task': {   'task1': tensor(0.000160, dtype=torch.float64, grad_fn=<DivBackward0>),
                'total': tensor(0.000160, dtype=torch.float64, grad_fn=<MulBackward0>)},
    'task_mean': {   'task1': tensor(3.338073e-06, dtype=torch.float64, grad_fn=<DivBackward0>),
                     'total': tensor(3.338073e-06, dtype=torch.float64, grad_fn=<MulBackward0>)},
    'total': {   'backprop': tensor(0.000160, dtype=torch.float64, grad_fn=<MulBackward0>),
                 'policy': tensor(0.000338),
      

In [ ]:
(environ.val_data['task1']['yc_data'][0] == environ.val_data['task1']['yc_data']).all()

In [ ]:
from utils.sparsechem_utils import compute_metrics, aggregate_results
import pandas
cc = compute_metrics(cols   = environ.val_data['task1']['yc_ind'][1], 
                     y_true = environ.val_data['task1']['yc_data'], 
                     y_score= environ.val_data['task1']['yc_hat'] ,
                     num_tasks=100)


In [ ]:
 df   = pd.DataFrame({"task"   : environ.val_data['task1']['yc_ind'][1], 
                      "y_true" : environ.val_data['task1']['yc_data'],  
                      "y_score": environ.val_data['task1']['yc_hat']})

In [ ]:
for task, frame in df.groupby("task", sort=True):
    print(f" task {task}")
    print(frame.head(10))

In [ ]:
# df
df.groupby("task", sort=True).count()

In [ ]:
cc

In [ ]:
print(environ.batch_data['task1']['yc_aggr_weights'])
environ.batch['task1']['aggr_weights']

In [ ]:
c2 = aggregate_results(cc)

In [ ]:
dldrs.trainset0.tasks_weights_list

#### display parms

In [ ]:
print( f" Backbone Initial LR         : {environ.opt['train']['backbone_lr']:4f}      current LR : {environ.optimizers['alphas'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR         : {environ.opt['train']['task_lr']:4f}      current LR : {environ.optimizers['weights'].param_groups[0]['lr']}    \n"
       f" Policy   Initial LR         : {environ.opt['train']['policy_lr']:4f}      current LR : {environ.optimizers['weights'].param_groups[1]['lr']}  \n")
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

In [ ]:

# environ.opt['train']['policy_lr'] = 0.01
# opt['train']['policy_lr']         = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['weights'].param_groups)
# print('current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
# print('current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
# print('current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

## Weight & Policy Training

### Weight/Policy Training Preparation

In [ ]:
# ns.flag_warmup = True
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
# ns.num_train_layers = None

In [ ]:
if ns.flag_warmup:
    print_heading( f"** {timestring()} \n"
                   f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
#     environ.define_optimizer(policy_learning=True)
#     environ.define_scheduler(policy_learning=True)
    ns.flag_warmup = False
    ns.flag = 'update_weights'
    environ.fix_alpha()
    environ.free_weights(opt['fix_BN'])

In [ ]:
# ns.training_epochs = 250
# environ.display_trained_policy(ns.current_epoch)
# environ.display_trained_logits(ns.current_epoch)

In [ ]:
print(f"ns.current_epoch           : {ns.current_epoch}")
print(f"ns.training_epochs         : {ns.training_epochs} \n") 
print(f"ns.current_iters           : {ns.current_iter}")  
print(f"Batches in weight epoch    : {ns.stop_iter_w}")
print(f"Batches in policy epoch    : {ns.stop_iter_a}")
print(f"num_train_layers           : {ns.num_train_layers}")
print()
print_loss(environ.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter}")
print()

print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

### Weight/Policy Training

In [ ]:
# weight_policy_training(ns, opt, environ, dldrs, epochs = 100)
weight_policy_training(ns, opt, environ, dldrs)

In [ ]:
ns.best_epoch, ns.best_iter, ns.best_value

### Close WandB run

In [ ]:
wandb.finish()

In [ ]:
# ns.best_epoch = 0
# from utils.notebook_modules import wrapup_phase
# wrapup_phase(ns, opt, environ)

In [ ]:
# environ.opt['train']['policy_lr']       = 0.002
# environ.opt['train']['lambda_sparsity'] = 0.05
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")

print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" 
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}\n") 

print( f" current_iters               : {ns.current_iter}   \n"
       f" current_epochs              : {ns.current_epoch}  \n" 
       f" train_total_epochs          : {ns.training_epochs}\n" 
       f" stop_epoch_training         : {ns.stop_epoch_training}")

## Post Training Stuff

In [ ]:
pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)
# environ.num_layers, environ.networks['mtl-net'].num_layers
# environ.batch

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
# metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
# environ.save_checkpoint(model_label, current_iter, current_epoch) 
# save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [ ]:
# environ.losses
# environ.val_metrics

In [ ]:
# environ.batch_data
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])
# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [ ]:
# environ.display_trained_logits(ns.current_epoch)
# environ.display_trained_policy(ns.current_epoch)

# environ.display_test_sample_policy(ns.current_epoch, hard_sampling = True)
# environ.display_train_sample_policy(ns.current_epoch, hard_sampling = True)

In [ ]:
# environ.define_optimizer(policy_learning=True)

In [ ]:
# print(environ.optimizers['alphas'])
# print(environ.optimizers['weights'])

In [ ]:
# print('Policy  initial_lr : ', environ.optimizers['alphas'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['alphas'].param_groups[0]['lr'])
# print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[0]['lr'])
# print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[1]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[1]['lr'])

In [ ]:
wandb.run is None

In [ ]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

In [ ]:
# wandb.finish()

In [6]:
from utils import load_sparse
ecfp     = load_sparse(opt['dataload']['dataroot'], opt['dataload']['x'])
folding  = np.load(os.path.join(opt['dataload']['dataroot'], opt['dataload']['folding']))

print(ecfp.shape, folding.shape)

fold_va = opt['dataload']['fold_va']
idx_tr  = np.where(folding != fold_va)[0]
idx_va  = np.where(folding == fold_va)[0]

(18388, 85277) (18388,)


In [9]:
folding[:50]

array([4, 0, 0, 3, 1, 2, 4, 0, 1, 0, 4, 2, 4, 3, 0, 0, 2, 4, 0, 4, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4,
       4, 4, 0])

In [10]:
idx_tr[:20]

array([ 0,  3,  4,  5,  6,  8, 10, 11, 12, 13, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26])

In [11]:
idx_tr2 = np.isin(folding, [1,2,3,4])

In [12]:
idx_tr2[:20]

array([ True, False, False,  True,  True,  True,  True, False,  True, False,  True,  True,  True,  True, False, False,  True,  True, False,  True])

In [15]:
ecfp_tr = ecfp[idx_tr]
ecfp_tr2 = ecfp[idx_tr2]

In [19]:
ecfp_tr2.shape

(14633, 85277)

In [42]:
res = (ecfp_tr != ecfp_tr2)
print(res.shape)
print(res[:10,:10].nnz)

(14633, 85277)
0


In [35]:
equal = 0
for i in range(14633):
    for j in range(1):
        if (ecfp_tr[i,j] != ecfp_tr2[i,j]):
            print(i, j, '     ', ecfp_tr[i,j], '    !=    ', ecfp_tr2[i,j])
        else:
            equal+=1 
print("total eaqual = ", equal)
# print(ecfp_tr2[0,:100])
# print(ecfp_tr[:10,:10] != ecfp_tr2[:10,:10])

total eaqual =  14633


### Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


### Post Weight + Policy Training Stuff 

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

### Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

### Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)